# DIVA

## A data driven research project on creating an audio-visual interactive map on diversity in the city of Vienna

### Importing Libaries

In [1]:
import pandas as pd
import folium
import geopandas as gpd
import numpy as np
import sounddevice as sd
import wavio

### Writing some Soundgenerators ##

In [2]:
# midi to frequency utility


def mtof(midi):
    frequency = 2 ** ((midi - 69) / 12) * 440
    return frequency

In [3]:
# simple panning algorithm

def panner(x, angle):
            left = np.sqrt(2)/2.0 * (np.cos(angle) - np.sin(angle)) * x
            right = np.sqrt(2)/2.0 * (np.cos(angle) + np.sin(angle)) * x
            return np.dstack((left, right))[0]

In [4]:
# waves and envelopes


global sr
sr = 44100


def sine(dur, freq):
    t = np.arange(0, dur * sr) / sr
    sine = np.sin(2 * freq * t * np.pi)
    return sine


def env(dur, at, dec):
    at = np.linspace(0, 1, int(at * dur * sr))
    dec = np.linspace(1, 0, int(dec * dur * sr))
    env = np.concatenate((at, dec))
    return env


### Composing 4 soundscapes 

In [5]:
# Scape 1

dur = 10


sound1_1 = sine(dur, mtof(48)) * sine(dur, 0.25) * env(dur, 0.1, 0.9)
sound1_2 = sine(dur, mtof(58)) * sine(dur, sine(dur, 0.04) * 2.5) * env(dur, 0.1, 0.9)
sound1_3 = sine(dur, mtof(62)) * sine(dur, sine(dur, 0.03) * 1.5) * env(dur, 0.1, 0.9)


# demo composition


rec_1 = (panner(sound1_1, -45) + panner(sound1_2, 35) 
         + panner(sound1_3, -25)) * 0.15

In [6]:
# pre-listening 1

sd.play(rec_1)

In [7]:
# Scape 2


sound2_1 = sine(dur, mtof(48)) * sine(dur, 0.25) * env(dur, 0.1, 0.9)
sound2_2 = sine(dur, mtof(58)) * sine(dur, 0.5) * env(dur, 0.1, 0.9)
sound2_3 = sine(dur, mtof(62)) * sine(dur, sine(dur, 0.03) * 1.5) * env(dur, 0.1, 0.9)
sound2_4 = sine(dur, mtof(72)) * sine(dur, sine(dur, 0.05) * 1.25) * env(dur, 0.1, 0.9)


rec_2 = (panner(sound2_1, -45) + panner(sound2_2, 35) 
         + panner(sound2_3, -25) + panner(sound2_4, 25)) * 0.15

In [8]:
# pre-listening 2

sd.play(rec_2)

In [9]:
# Scape 3


sound3_1 = sine(dur, mtof(48)) * sine(dur, 0.25) * env(dur, 0.1, 0.9)
sound3_2 = sine(dur, mtof(58)) * sine(dur, 0.5) * env(dur, 0.1, 0.9)
sound3_3 = sine(dur, mtof(62)) * sine(dur, sine(dur, 0.03) * 1.5) * env(dur, 0.1, 0.9)
sound3_4 = sine(dur, mtof(72)) * sine(dur, sine(dur, 0.05) * 1.25) * env(dur, 0.1, 0.9)
sound3_5 = sine(dur, mtof(77)) * sine(dur, sine(dur, 0.05) * 1.75) * env(dur, 0.1, 0.9)


rec_3 = (panner(sound3_1, -45) + panner(sound3_2, 35) 
         + panner(sound3_3, -25) + panner(sound3_4, 25)
         + panner(sound3_5, 0) * 0.4) * 0.15

In [10]:
# pre-listening 3

sd.play(rec_3)

In [11]:
# Scape 4


sound4_1 = sine(dur, mtof(48)) * sine(dur, 0.25) * env(dur, 0.1, 0.9)
sound4_2 = sine(dur, mtof(67)) * sine(dur, 0.5) * env(dur, 0.1, 0.9)
sound4_3 = sine(dur, mtof(65)) * sine(dur, sine(dur, 0.03) * 1.5) * env(dur, 0.1, 0.9)
sound4_4 = sine(dur, mtof(74)) * sine(dur, 0.7) * env(dur, 0.1, 0.9)
sound4_5 = sine(dur, mtof(77)) * sine(dur, sine(dur, 0.03) * 1.25) * env(dur, 0.1, 0.9)
sound4_6 = sine(dur, mtof(86)) * sine(dur, sine(dur, 0.07) * 2.25) * env(dur, 0.1, 0.9)

rec_4 = (panner(sound4_1, -45) + panner(sound4_2, 35) 
         + panner(sound4_3, -25) + panner(sound4_4, 25)
         + panner(sound4_5, -5) 
         + panner(sound4_6, 5) * 0.5) * 0.13

In [12]:
# pre-listening 4

sd.play(rec_4)

In [13]:
# write to wav_file

recs = [rec_1, rec_2, rec_3, rec_4]

for i, rec in enumerate(recs):
    wavio.write(f'rec_{i + 1}.wav', rec, sr, sampwidth=2)

### Getting the data ready

In [14]:
# importing kml and xlsx files

gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
df = gpd.read_file('AllPointsAllData.kml', driver='KML')

df_2 = pd.read_excel('AllPointsAllData.xlsx')

In [15]:
# joining the datasets

df_3 = pd.concat([df, df_2], axis=1)

In [16]:
# selecting only 1 row per district - for demonstration purposes

df_4 = df_3.drop_duplicates(subset='BEZ')
df_4 = df_4.dropna().reset_index()

In [17]:
# getting lat and lon values ready for use with folium

points = [df_4.geometry[x].bounds[:2] for x in range(len(df_4.geometry))]
points2 = [[y[1], y[0]] for y in points]

In [18]:
# Setting up a new column with percentage of non-Austrian-inhabitants


for i in range(len(df_4['PCT(POP_AU'])):
    df_4.loc[i, 'perc_non_aut'] = 100 - df_4.loc[i, 'PCT(POP_AU']


In [19]:
# setting up a new row for the sum of percenteges of left parties - only rule: more than 0% in more than 19 districts

for i in range(len(df_4['PCT_SPOE'])):
    df_4.loc[i, 'perc_left_par'] = sum([float(j) for j in df_4.loc[i, ['PCT_SPOE', 'PCT_GRUE', 'PCT_LINKS']]])


In [20]:
df_4.iloc[:, -2:]

,perc_non_aut,perc_left_par
0,37.519623,44.966443
1,42.228912,74.384236
2,37.383011,59.400545
3,44.720236,58.536585
4,38.657128,57.512953
5,37.323760,62.083333
6,37.959927,61.052632
7,42.566595,65.779468
8,41.869118,60.641399
9,49.945319,65.365854


In [21]:
# summing the "diversity"-columns

for i in range(len(df_4['perc_non_aut'])):
    df_4.loc[i, 'div_param'] = sum([float(j) for j in df_4.loc[i, ['perc_non_aut', 'perc_left_par']]])


In [22]:
# checking min and max values

min_div = min(df_4['div_param'])
max_div = max(df_4['div_param'])

# creating segments

segment = (max_div - min_div) / 4

In [23]:
m = folium.Map(location=[48.20, 16.40], tiles='OpenStreetMap', zoom_start=11)


for x in range(len(df_4)):
    info = df_4.iloc[x, -1]
    if df_4.iloc[x, -1] < (min_div + segment):
        soundsrc = 'rec_1.wav'
    elif (min_div + segment) < df_4.iloc[x, -1] < (min_div + 2 * segment):
        soundsrc = 'rec_2.wav'
    elif (min_div + 2 * segment) < df_4.iloc[x, -1] < (min_div + 3 * segment):
        soundsrc = 'rec_3.wav'
    elif (min_div + 3 * segment) < df_4.iloc[x, -1] <= max_div:
        soundsrc = 'rec_4.wav'
    folium.CircleMarker(
    location=points2[x],
    radius=7,
    popup='</iframe>' \
            f'Diversity - Factor of {info} \
            <br><br> \
            <audio controls src={soundsrc} scrolling="no" \
            frameborder="0" width="340" height="115"></audio>' \
            '</iframe>',
    color="#F8F8FF",
    fill=True,
    fill_color="#3186cc",
).add_to(m)
    
m